In [17]:
from numpy import *
from time import sleep
import os
import errno
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_normal
from keras.utils import np_utils
#from keras_sequential_ascii import sequential_model_to_ascii_printout
from sklearn.model_selection import train_test_split

In [18]:
import tensorflow as tf

In [19]:
cd /Users/sagarsahoo/Downloads/

/Users/sagarsahoo/Downloads


In [20]:
def silent_remove(filename):
    try:
        os.remove(filename)
    except OSError as e:  # this would be "except OSError, e:" before Python 2.6
        if e.errno != errno.ENOENT:  # errno.ENOENT = no such file or directory
            raise  # re-raise exception if a different error occurred



def unpickle(file):
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='bytes')
    return d


def read_data(folder):
    x_data_temp = []
    y_data_temp = []
    x_test_data_temp = []
    y_test_data_temp = []
    # We don't use numpy's vstack here as that would be wasteful, because every time you do a vstack, numpy would end
    # up copying the whole array to a new location. Hence we use a little trick to first store the data in a list and
    # then convert it to an numpy array

    for file in os.listdir(folder):
        if file.endswith(".meta") or file.endswith(".html"):
            print("Ignoring html and meta files")
        elif "test_batch" in file:
            # test data file detected. we are gonna load it separately
            test_data_temp = unpickle(folder + "/" + file)
            x_test_data_temp.append(test_data_temp[b'data'])
            y_test_data_temp.append(test_data_temp[b'labels'])
        else:
            temp_data = unpickle(folder + "/" + file)
            x_data_temp.append(temp_data[b'data'])
            y_data_temp.append(temp_data[b'labels'])
    x_data = array(x_data_temp)
    y_data = array(y_data_temp)
    x_test_data = array(x_test_data_temp)
    y_test_data = array(y_test_data_temp)
    return [x_data, y_data, x_test_data, y_test_data]

In [21]:
X_train_temp, y_train_temp, X_test_temp, y_test_temp = read_data("cifar-10-batches-py")

Ignoring html and meta files
Ignoring html and meta files


In [22]:
print(X_train_temp.shape, X_train_temp.ndim, type(X_train_temp))
print(y_train_temp.shape, y_train_temp.ndim, type(y_train_temp))

(5, 10000, 3072) 3 <class 'numpy.ndarray'>
(5, 10000) 2 <class 'numpy.ndarray'>


In [23]:
X_train_temp = X_train_temp.reshape(X_train_temp.shape[0] * X_train_temp.shape[1], X_train_temp.shape[2])
y_train_temp = y_train_temp.reshape(y_train_temp.shape[0] * y_train_temp.shape[1])

# Similarly for X_test_temp and y_test_data

X_test_temp = X_test_temp.reshape(X_test_temp.shape[0] * X_test_temp.shape[1], X_test_temp.shape[2])
y_test_temp = y_test_temp.reshape(y_test_temp.shape[0] * y_test_temp.shape[1])

print(X_train_temp.shape, X_train_temp.ndim, type(X_train_temp))
print(y_train_temp.shape, y_train_temp.ndim, type(y_train_temp))

print(X_test_temp.shape, X_test_temp.ndim, type(X_test_temp))
print(y_test_temp.shape, y_test_temp.ndim, type(y_test_temp))


(50000, 3072) 2 <class 'numpy.ndarray'>
(50000,) 1 <class 'numpy.ndarray'>
(10000, 3072) 2 <class 'numpy.ndarray'>
(10000,) 1 <class 'numpy.ndarray'>


In [24]:
X_train, y_train = shuffle(X_train_temp, y_train_temp, random_state=4)
X_test, y_test = shuffle(X_test_temp, y_test_temp, random_state=4)

In [25]:
# Splitting X and y in training and val data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=13433382)

# PCA

In [26]:
from sklearn.decomposition import PCA

In [27]:
pca = PCA()
pca.fit_transform(X_train)

array([[ 2.65038275e+03, -1.95174099e+03,  3.17917466e+01, ...,
         5.46725631e-01, -2.66012072e-01,  5.11939338e-01],
       [-2.16472184e+02,  1.61471474e+03,  1.61462560e+03, ...,
         1.39951359e-01,  4.25294685e-01,  6.76194147e-04],
       [ 1.89235266e+03, -2.90273939e+03,  7.26465489e+02, ...,
         2.26548715e-02,  1.07499696e-01, -5.30930803e-01],
       ...,
       [-1.88527631e+02,  7.40872823e+02,  3.70187083e+02, ...,
         2.49327761e-01,  2.43933006e-01,  1.28979018e-01],
       [-1.35009278e+03,  2.93335128e+02, -6.68738936e+02, ...,
         1.22121856e-01, -2.67951781e-01, -8.70771836e-02],
       [-1.88531819e+02,  7.94615150e+02,  3.13328845e+02, ...,
        -1.53393912e-01,  5.40372176e-02,  7.22672867e-02]])

In [28]:
print('Explained Variance: ', pca.explained_variance_.shape)

k = 0
total = sum(pca.explained_variance_)
current_sum = 0

while(current_sum / total < 0.99):
    current_sum += pca.explained_variance_[k]
    k += 1
    
print('k:  ',k)


## Applying PCA with k calcuated above

pca = PCA(n_components=k, whiten=True)

x_train_pca = pca.fit_transform(X_train)
x_val_pca = pca.transform(X_val)


print(x_train_pca.shape)
print(y_train.shape)
print(x_val_pca.shape)
print(y_val.shape)

Explained Variance:  (3072,)
k:   657
(40000, 657)
(40000,)
(10000, 657)
(10000,)


# DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
dtc = DecisionTreeClassifier(random_state=0,max_depth=60,min_samples_split=80)

# fit the grid with data
dtc_grid.fit(x_train_pca, y_train)

In [ ]:
y_dtc_val_pred = dtc_grid.best_estimator_.predict(x_val_pca)
y_dtc_insample_pred = dtc_grid.best_estimator_.predict(x_train_pca)


print('DecisionTreeClassifier Validation Accuracy : ',metrics.accuracy_score(y_dtc_val_pred,y_val))
print('DecisionTreeClassifier In-Sample Accuracy : ',metrics.accuracy_score(y_dtc_insample_pred,y_train))

# RandomForestClassifier

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

1.0

In [ ]:
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(random_state=0)

# fit the grid with data
rfc.fit(x_train_pca, y_train)

In [ ]:
y_rfc_val_pred = rfc.predict(x_val_pca)
y_rfc_insample_pred = rfc.predict(x_train_pca)


print('RandomForestClassifier Validation Accuracy : ',metrics.accuracy_score(y_rfc_val_pred,y_val))
print('RandomForestClassifier In-Sample Accuracy : ',metrics.accuracy_score(y_rfc_insample_pred,y_train))

# Gradient Boosting

In [ ]:
GBC = GradientBoostingClassifier()
GBC.fit(x_train_pca,y_train)

y_val_gbc_pred=GBC.predict(x_val_pca)
y_gbc_insample_pred=GBC.predict(x_val_pca)


print('Gradient Boosting Validation Accuracy : ' ,accuracy_score(y_val, y_val_gbc_pred))
print('Gradient Boosting Training Accuracy : ',accuracy_score(y_gbc_insample_pred,y_train))

# Support Vector Machines (SVM - RBF Kernel)

In [ ]:
from sklearn.svm import SVC

In [ ]:
rbf_svclassifier = SVC(kernel='rbf',probability=True)
rbf_svclassifier.fit(x_train_pca, y_train)

y_val_rbf_pred = rbf_svclassifier.predict(x_val_pca)
y_insample_rbf_pred = rbf_svclassifier.predict(x_train_pca)

print( 'RBF SVM Validation Accuracy : ', accuracy_score(y_val, y_val_rbf_pred))
print('RBF SVM In-Sample Accuracy : ', accuracy_score(y_train, y_insample_rbf_pred))